<a href="https://colab.research.google.com/github/RebecaGis/Py/blob/main/Geocodifica%C3%A7%C3%A3o_Completa_Rua%2C_N%C3%BAmero%2C_Bairro%2C_Cidade_e_CEP_OSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy
import requests
import time

# Configurações
shapefile = r"C:CAMINHO/DO/ARQUIVO/AQUI"
campo_lat = "Latitude"  # substitua pelo nome real do campo
campo_lon = "Longitude"  # substitua pelo nome real do campo

# Campos de endereço que serão preenchidos
campos_endereco = {
    "Rua": "road",
    "Número": "house_number",
    "Bairro": ["suburb", "neighbourhood"],
    "Cidade": ["city", "town"],
    "CEP": "postcode"
}

# Verificar e criar os campos se necessário
for nome_campo in campos_endereco:
    if nome_campo not in [field.name for field in arcpy.ListFields(shapefile)]:
        arcpy.AddField_management(shapefile, nome_campo, "TEXT", field_length=100)

# Função para geocodificação reversa usando Nominatim (OpenStreetMap)
def get_address_components(lat, lon):
    try:
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1"
        headers = {'User-Agent': 'PDAU-Geocoding/1.0'}
        response = requests.get(url, headers=headers).json()
        address = response.get('address', {})

        dados = {
            "Rua": address.get("road") or address.get("pedestrian", ""),
            "Número": address.get("house_number", ""),
            "Bairro": address.get("suburb") or address.get("neighbourhood", ""),
            "Cidade": address.get("city") or address.get("town", ""),
            "CEP": address.get("postcode", "")
        }

        return dados
    except Exception as e:
        print(f"Erro ao geocodificar ({lat}, {lon}): {str(e)}")
        return {k: "Erro" for k in campos_endereco}

# Atualizar todas as features
campos_cursor = [campo_lat, campo_lon] + list(campos_endereco.keys())

with arcpy.da.UpdateCursor(shapefile, campos_cursor) as cursor:
    contador = 0
    for row in cursor:
        lat, lon = row[0], row[1]
        if lat and lon:
            print(f"Processando feature {contador+1}...")
            dados_endereco = get_address_components(lat, lon)
            for i, campo in enumerate(campos_endereco.keys(), start=2):
                row[i] = dados_endereco[campo]
            cursor.updateRow(row)
            contador += 1
            time.sleep(1.2)  # Limite da Nominatim
        else:
            print(f"Coordenadas ausentes na feature {contador+1}")
            for i in range(2, len(row)):
                row[i] = "Coordenadas ausentes"
            cursor.updateRow(row)
            contador += 1

print(f"\nProcesso concluído! {contador} feições foram atualizadas.")
